In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

In [ ]:
livres_schema = StructType([
    StructField("livre_id", StringType(), False),
    StructField("titre", StringType(), False),
    StructField("prix", IntegerType(), False),
    StructField("auteur_id", StringType(), True)
])

livres_data = [
    ("L1", "Titre 1", 45, "A1"),
    ("L2", "Titre 2", 16, "A2"),
    ("L3", "Titre 3", 15, "A1"),
    ("L4", "Titre 4", 24, "A3"),
    ("L5", "Titre 5", 27, None)
]

df_livres = spark.createDataFrame(livres_data, livres_schema)

In [6]:
df_livres.show()

+--------+-------+----+---------+
|livre_id|  titre|prix|auteur_id|
+--------+-------+----+---------+
|      L1|Titre 1|  45|       A1|
|      L2|Titre 2|  16|       A2|
|      L3|Titre 3|  15|       A1|
|      L4|Titre 4|  24|       A3|
|      L5|Titre 5|  27|     NULL|
+--------+-------+----+---------+



In [7]:
auteurs_schema = StructType([
    StructField("auteur_id", StringType(), False),
    StructField("nom", StringType(), False),
    StructField("prenom", StringType(), False)
])

auteurs_data = [
    ("A1", "Toto", "Toto"),
    ("A2", "Tata", "Tata"),
    ("A3", "Titi", "Titi"),
    ("A4", "Tutu", "Tutu")
]

df_auteurs = spark.createDataFrame(auteurs_data, auteurs_schema)

In [8]:
df_auteurs.show()

+---------+----+------+
|auteur_id| nom|prenom|
+---------+----+------+
|       A1|Toto|  Toto|
|       A2|Tata|  Tata|
|       A3|Titi|  Titi|
|       A4|Tutu|  Tutu|
+---------+----+------+



In [10]:
print("INNER JOIN - Seulement les livres qui ont un auteur")
new_df = df_livres.join(df_auteurs, "auteur_id", "inner")
new_df.show()

INNER JOIN - Seulement les livres qui ont un auteur
+---------+--------+-------+----+----+------+
|auteur_id|livre_id|  titre|prix| nom|prenom|
+---------+--------+-------+----+----+------+
|       A1|      L1|Titre 1|  45|Toto|  Toto|
|       A1|      L3|Titre 3|  15|Toto|  Toto|
|       A2|      L2|Titre 2|  16|Tata|  Tata|
|       A3|      L4|Titre 4|  24|Titi|  Titi|
+---------+--------+-------+----+----+------+



In [ ]:
print("LEFT JOIN - Tous les livres, et les auteur qui ont un livre")
# new_df2 = df_livres.join(df_auteurs, "auteur_id", "inner")
new_df2 = df_livres.join(df_auteurs, df_auteurs.auteur_id==df_livres.auteur_id , "left")
new_df2.show()

LEFT JOIN - Tous les livres, et les auteur qui ont un livre
+---------+--------+-------+----+----+------+
|auteur_id|livre_id|  titre|prix| nom|prenom|
+---------+--------+-------+----+----+------+
|       A2|      L2|Titre 2|  16|Tata|  Tata|
|     NULL|      L5|Titre 5|  27|NULL|  NULL|
|       A3|      L4|Titre 4|  24|Titi|  Titi|
|       A1|      L1|Titre 1|  45|Toto|  Toto|
|       A1|      L3|Titre 3|  15|Toto|  Toto|
+---------+--------+-------+----+----+------+



In [13]:
print("ANTI JOIN - Livres sans auteur")
new_df3 = df_livres.join(df_auteurs, "auteur_id", "anti")
new_df3.show()

ANTI JOIN - Livres sans auteur
+---------+--------+-------+----+
|auteur_id|livre_id|  titre|prix|
+---------+--------+-------+----+
|     NULL|      L5|Titre 5|  27|
+---------+--------+-------+----+

